In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!git clone https://github.com/logacc99/LPR.git

Cloning into 'LPR'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 267 (delta 46), reused 254 (delta 41), pack-reused 0
Receiving objects: 100% (267/267), 1.05 MiB | 5.38 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
import os
os.symlink('/content/drive/MyDrive/lpr-assets/models','/content/LPR/models')

In [2]:
%cd /content/LPR

/content/LPR


In [6]:
!rm -rf /content/LPR/models

In [ ]:
!pip install -qr requirements.txt # install

In [6]:
%load_ext autoreload
%autoreload 2
from full_flow import detect_batch, LP_Detector

From /content/LPR/full_flow.py:17: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True


In [7]:
lp_detector = LP_Detector()

Loading WPOD-NET...


Fusing layers... 


Loading Vehicle Detector...


Model Summary: 213 layers, 7225885 parameters, 0 gradients


Loading OCR Model...
Loading Type Predictor...
Initialize done.


In [4]:
from glob import glob
import cv2
import numpy as np
img_paths = glob('/content/drive/MyDrive/UFPR-ALPR-dataset/testing/**/*.png', recursive=True)
# img_paths = glob('/content/*.png', recursive=True)
img_paths = [path for path in img_paths if any([key in path for key in ['[01]','[10]','[20]','[30]']])]
img_paths = [path for path in img_paths if '111[20]' in path]
print(len(img_paths))
print(img_paths[0])

1
/content/drive/MyDrive/UFPR-ALPR-dataset/testing/track0111/track0111[20].png


In [ ]:
np.random.shuffle(img_paths)

In [5]:
save_dir = '/content/drive/MyDrive/yolov5/output/testing-9-1'
detect_batch(img_paths, lp_detector, save_dir = '/content/output')

  0%|          | 0/1 [00:00<?, ?it/s]

(240, 80)
AIQP1056


100%|██████████| 1/1 [00:21<00:00, 21.89s/it]


# Test accuracy

In [ ]:
%rm -rf /content/testing

In [ ]:
!unzip /content/testing.zip -d /content/testing

In [ ]:
from glob import glob
import cv2
import numpy as np
import editdistance
gt_paths = glob('/content/drive/MyDrive/UFPR-ALPR-dataset/labels/ocr/testing/**/*.txt', recursive=True)
gt_paths = [path for path in gt_paths if any([key in path for key in ['[01]', '[05]', '[10]','[15]' ,'[20]', '[25]', '[30]']])]
pred_paths = glob('/content/output/**/*.txt', recursive=True)
print(len(gt_paths), len(pred_paths))

In [ ]:
total = len(pred_paths)
from tqdm import tqdm
wrong_detection = []
wrong_count = 0
avg_acc = 0
for pred_path in tqdm(pred_paths):
    fname = pred_path.split('/')[-1]
    gt_path = '/content/drive/MyDrive/UFPR-ALPR-dataset/labels/ocr/testing/' + fname
    with open(gt_path, 'r') as f:
        anno = f.readline()
        gt_ocr = anno.split(' ')[-1]
        # print('GT:', gt_ocr)
    
    with open(pred_path, 'r') as f:
        pred_ocr = f.read()
        # print('Pred:', pred_ocr)

    gt_ocr = gt_ocr.replace('-','')
    if pred_ocr != 'None':
        pred_ocr = pred_ocr[:-3]

    if pred_ocr != gt_ocr:
        wrong_count +=1
        wrong_detection.append([fname, gt_ocr, pred_ocr])
    
    # Another metrics: Edit Distance
    ed = editdistance.eval(gt_ocr, pred_ocr)
    acc = 1-ed/len(gt_ocr)
    if acc < 0: acc = 0
    avg_acc += acc / len(gt_paths)

In [ ]:
print('Wrong detections:', wrong_count)
print('Accuracy:', (total-wrong_count)/total)
print(avg_acc)

In [ ]:
wrong_detection = sorted(wrong_detection, key = lambda x: x[0])
for det in wrong_detection:
    print(det)